<a href="https://colab.research.google.com/github/Mrmesmerized/Toxic-Word-Detection-Project/blob/main/notebooks/TWD_preview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
# Install gdown to download files from Google Drive
!pip install gdown

# Download the dataset from Google Drive using the file ID
import gdown

file_id = '1RbAtx5t7x7Com2sjEG6bLrXYJy3LI0vA'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'twitter_data.csv'
gdown.download(url, output, quiet=False)

# Load the dataset
import pandas as pd

df = pd.read_csv('twitter_data.csv')

# Display the first few rows of the dataset
df.head()

Downloading...
From: https://drive.google.com/uc?id=1RbAtx5t7x7Com2sjEG6bLrXYJy3LI0vA
To: /content/twitter_data.csv
100%|██████████| 2.55M/2.55M [00:00<00:00, 47.1MB/s]


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import re
import nltk
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer('english')
from nltk.corpus import stopwords
import string
stopword = set(stopwords.words('english'))

# Function to load and prepare the data
def load_and_prepare_data(filepath):
    df = pd.read_csv('/content/twitter_data.csv')

    # Map the 'class' column to 'labels'
    df['labels'] = df['class'].map({0: "Hate Speech Detected", 1: "Offensive language detected", 2: "No hate and offensive speech"})

    # Retain a copy of the 'class' column for later filtering
    class_column = df['class'].copy()

    # Select relevant columns
    df = df[['tweet', 'labels', 'class']]  # Keep 'class' for filtering later

    return df, class_column

# Clean the text data
def clean(text):
    text = str(text).lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = [word for word in text.split() if word not in stopword]
    text = " ".join(text)
    text = [stemmer.stem(word) for word in text.split()]
    text = " ".join(text)
    return text

# Load the data
df, class_column = load_and_prepare_data(r'/content/twitter_data.csv')

# Apply cleaning
df["tweet"] = df["tweet"].apply(clean)

# Drop rows with missing values
df = df.dropna(subset=['tweet', 'labels'])

# Convert the text data to arrays
x = np.array(df["tweet"])
y = np.array(df["class"])  # Use 'class' column for training

# Vectorize the text data
cv = CountVectorizer()
x_cv = cv.fit_transform(x)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_cv, y, test_size=0.33, random_state=42)

# Train the decision tree classifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Define the labels mapping
labels_mapping = {0: "Hate Speech Detected", 1: "Offensive language detected", 2: "No hate and offensive speech"}

# Create the text box widget with increased width
text_box = widgets.Textarea(
    value='',
    placeholder='Type your text here...',
    description='Text:',
    disabled=False,
    layout=widgets.Layout(width='80%')  # Set the width to 80% of the available space
)

# Create the button widget with vertical alignment
button = widgets.Button(
    description='Detect',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to detect',
    icon='check',
    layout=widgets.Layout(margin='20px 30px 30px 60px')  # Adjust the margin to align vertically
)

output = widgets.Output()

def on_button_click(b):
    with output:
        try:
            output.clear_output()
            input_text = text_box.value
            cleaned_text = clean(input_text)
            input_vec = cv.transform([cleaned_text])
            prediction = clf.predict(input_vec)[0]
            result = labels_mapping[prediction]
            print("Detection:", result)
        except Exception as e:
            print(f"Error: {e}")

button.on_click(on_button_click)

# Align the button and text box vertically
vbox = widgets.VBox([text_box, button])

display(vbox, output)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Output()